In [1]:
import konlpy
from konlpy.tag import Twitter
import jpype
Twitter()

In [1]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB , GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from konlpy.tag import Twitter
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
pos_tagger = Twitter()
# read train data

with open('./data/train_data.json') as fp:
    json_str = fp.read()
    json_data = json.loads(json_str)
    
# convert to dataframe
train_df = pd.DataFrame(json_data)

# train data preprocessing
train_df['rating_cat'] = train_df['rating'].apply(lambda x: 
                    'NEG' if 1<= x <=3 
                     else 
                      ('NEU' if 4<=x<=7 
                     else 'POS'))



input_file = './data/test.input'

with open(input_file) as fp:
    test_in = fp.read()

test_in = test_in.splitlines()


with open('./data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()


In [2]:
%timeit
from konlpy.tag import Twitter
twitter = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), use_idf=False, smooth_idf=False)


y = train_df.rating_cat
X = vectorizer.fit_transform(train_df.review)

In [10]:
model = LogisticRegression(C=30).fit(X, y)

In [14]:
from sklearn.ensemble import VotingClassifier
model1 = LinearSVC()
model2 = LogisticRegression()
model3 = SGDClassifier(loss='modified_huber',alpha=0.0000039)
ensemble = VotingClassifier(estimators=[('svc1', model1), ('logit', model2), ('svc2', model3)], 
                            voting='soft', weights=[2, 1, 1.5])

In [7]:
model = LinearSVC(C=0.5)
model.fit(X,y)

LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [11]:
from sklearn.externals import joblib
feature_list = vectorizer.get_feature_names()

input_file = './data/grading.input'

with open(input_file) as fp:
    test_in = fp.read()

test_in = test_in.splitlines()


# joblib.dump(feature_list, 'vocabulary.pkl')

# vocabulary = joblib.load('vocabulary.pkl') 
# vocabulary

test_vectorizer =  TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2), vocabulary = feature_list, use_idf=False, smooth_idf=False)
X_test = test_vectorizer.fit_transform(test_in[:8400])


test_pred = model.predict(X_test)

In [13]:
output_file = 'submission.txt'#sys.argv[2] # 


with open(output_file, 'w') as fp:
    for _test_pred in test_pred:
        fp.write(_test_pred)
        fp.write('\n')
        
#t(koken twit pos) - s
#print(classification_report(test_out[:8400],test_pred))
#print(accuracy_score(test_out[:8400], test_pred))


In [8]:
#alpha 최적값 찾기
alpha = np.arange(0.0000039,0.000004, 0.00000001)
alpha = np.arange(0.000001,0.00001,0.000001) ## 3e-6 이 일등
alpha = np.arange(2.5e-6,3.5e-6,1e-7) ## 2.9e-6이 일등
alpha = [0.001, 0.0001, 0.00001, 0.000001, 0.0000001]

ite = 2 #  반복
score = np.zeros([len(alpha), ite])

for i, val in enumerate(alpha):
    for k in range(ite): 
        model = SGDClassifier(alpha=val).fit(X, y)
        test_pred = model.predict(X_test)

        score[i][k] = accuracy_score(test_out[:8400], test_pred)
        
print(score)
print(score.mean(axis=1))

[[ 0.66107143  0.66107143]
 [ 0.66107143  0.66107143]
 [ 0.66107143  0.66107143]
 [ 0.65904762  0.65880952]
 [ 0.63285714  0.63369048]]
[ 0.66107143  0.66107143  0.66107143  0.65892857  0.63327381]


In [144]:
# n_iter 최적

n = np.arange(15,25)


ite = 5 #  반복
score = np.zeros([len(n), ite])


for i, val in enumerate(n):
    for k in range(ite): 
        
        model = SGDClassifier(alpha=2.9e-6, n_iter=val).fit(X, y)
        test_pred = model.predict(X_test)

        score[i][k] = accuracy_score(test_out[:8400], test_pred)
        
print(score)
print(score.mean(axis=1))

[[ 0.77797619  0.77595238  0.7777381   0.77666667  0.7775    ]
 [ 0.77833333  0.77690476  0.77738095  0.77654762  0.77690476]
 [ 0.7777381   0.7777381   0.77702381  0.77690476  0.77678571]
 [ 0.77761905  0.7775      0.7777381   0.77714286  0.77785714]
 [ 0.7777381   0.7775      0.77833333  0.77702381  0.77797619]
 [ 0.7772619   0.77833333  0.77642857  0.77678571  0.77642857]
 [ 0.7772619   0.7772619   0.77678571  0.7772619   0.77702381]
 [ 0.77738095  0.77809524  0.77785714  0.77654762  0.77797619]
 [ 0.77738095  0.77785714  0.77761905  0.7775      0.7777381 ]
 [ 0.77678571  0.77702381  0.77797619  0.7772619   0.77761905]]
[ 0.77716667  0.77721429  0.7772381   0.77757143  0.77771429  0.77704762
  0.77711905  0.77757143  0.77761905  0.77733333]


In [288]:
model = SGDClassifier(alpha=2.9e-6, n_iter=19).fit(X, y)

In [ ]:
X

In [9]:
#alpha 최적값 찾기
alpha = np.arange(0.0000039,0.000004, 0.00000001)
alpha = np.arange(0.000001,0.00001,0.000001) ## 3e-6 이 일등
alpha = np.arange(2.5e-6,3.5e-6,1e-7) ## 2.9e-6이 일등
alpha = [0.001, 0.0001, 0.00001, 0.000001, 0.0000001]

ite = 2 #  반복
score = np.zeros([len(alpha), ite])

for i, val in enumerate(alpha):
    for k in range(ite): 
        model = SGDClassifier(alpha=val).fit(X, y)
        test_pred = model.predict(X_test)

        score[i][k] = accuracy_score(test_out[:8400], test_pred)
        
print(score)
print(score.mean(axis=1))

[[ 0.66107143  0.66107143]
 [ 0.66107143  0.66107143]
 [ 0.66107143  0.66107143]
 [ 0.65952381  0.65809524]
 [ 0.63392857  0.63428571]]
[ 0.66107143  0.66107143  0.66107143  0.65880952  0.63410714]


In [127]:
score.max()

0.77892857142857141

In [129]:
alpha = np.arange(2.5e-6,3.5e-6,1e-7)
alpha

array([  2.50000000e-06,   2.60000000e-06,   2.70000000e-06,
         2.80000000e-06,   2.90000000e-06,   3.00000000e-06,
         3.10000000e-06,   3.20000000e-06,   3.30000000e-06,
         3.40000000e-06])

In [118]:
score

array([[ 0.76916667],
       [ 0.77690476],
       [ 0.77952381],
       [ 0.77642857],
       [ 0.77440476],
       [ 0.77369048],
       [ 0.7702381 ],
       [ 0.76988095],
       [ 0.7675    ]])

In [106]:
a=  np.array([[1,1],[2,2],[3,3]])
a
a.mean(axis=1)

array([ 1.,  2.,  3.])

In [101]:
score

array([[ 0.77654762,  0.77357143],
       [ 0.77571429,  0.77452381],
       [ 0.77571429,  0.77404762],
       [ 0.77595238,  0.77559524],
       [ 0.77428571,  0.77535714],
       [ 0.77559524,  0.77452381],
       [ 0.7752381 ,  0.77464286],
       [ 0.77380952,  0.77595238],
       [ 0.77511905,  0.77559524],
       [ 0.77440476,  0.77380952]])

In [100]:
score.mean(axis=1)

array([ 0.77505952,  0.77511905,  0.77488095,  0.77577381,  0.77482143,
        0.77505952,  0.77494048,  0.77488095,  0.77535714,  0.77410714])

In [76]:
alpha = np.arange(0.0000039,0.000004, 0.00000001)

for i in alpha:
    model = SGDClassifier(alpha=i).fit(X, y)
    test_pred = model.predict(X_test)

    print(accuracy_score(test_out[:8400], test_pred))

0.775833333333
0.775714285714
0.774642857143
0.774285714286
0.77619047619
0.776666666667
0.774880952381
0.775
0.774761904762
0.772976190476


In [75]:
alpha = np.arange(0.0000039,0.000004, 0.00000001)
a = (0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001)
for i in alpha:
    model = SGDClassifier(alpha=i).fit(X, y)
    test_pred = model.predict(X_test)

    print(accuracy_score(test_out[:8400], test_pred))

0.775119047619
0.776428571429
0.774166666667
0.775
0.775
0.775595238095
0.775119047619
0.775
0.77619047619
0.775


In [ ]:
alpha = (0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001)
for i in alpha:
    model = SGDClassifier(alpha=i).fit(X, y)
    test_pred = model.predict(X_test)

    print(accuracy_score(test_out[:8400], test_pred))

In [ ]:
model = SGDClassifier(alpha=0.0000039).fit(X, y)

In [60]:
test_pred = model.predict(X_test)

In [292]:
# test out sgd (3.9*e-6) #high score # 최대
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.63      0.56      0.59      1022
        NEU       0.67      0.39      0.49      1825
        POS       0.82      0.94      0.88      5553

avg / total       0.76      0.78      0.76      8400

0.777857142857
0.7855
0.7855
0.751
0.7875


In [61]:
# test out sgd (3.9*e-6) #high score # 최대
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.64      0.55      0.59      1022
        NEU       0.68      0.38      0.48      1825
        POS       0.81      0.95      0.87      5553

avg / total       0.76      0.78      0.76      8400

0.776547619048
0.7815
0.7845
0.747
0.790416666667


In [58]:
# test out sgd (3.5*e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.63      0.56      0.59      1022
        NEU       0.66      0.39      0.49      1825
        POS       0.81      0.94      0.87      5553

avg / total       0.76      0.78      0.76      8400

0.775238095238
0.7835
0.7815
0.7475
0.78625


In [55]:
# test out sgd (4*e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.63      0.55      0.59      1022
        NEU       0.67      0.37      0.48      1825
        POS       0.81      0.95      0.87      5553

avg / total       0.76      0.77      0.75      8400

0.774285714286
0.783
0.783
0.743
0.785833333333


In [52]:
# test out sgd (3*e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.64      0.56      0.60      1022
        NEU       0.66      0.37      0.48      1825
        POS       0.81      0.95      0.87      5553

avg / total       0.76      0.78      0.75      8400

0.775714285714
0.7815
0.7865
0.7455
0.787083333333


In [49]:
# test out sgd (7*e-5)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.65      0.53      0.58      1022
        NEU       0.68      0.34      0.46      1825
        POS       0.80      0.96      0.87      5553

avg / total       0.76      0.77      0.75      8400

0.77130952381
0.7785
0.782
0.743
0.78


In [45]:
# test out sgd (5*e-5)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.64      0.55      0.59      1022
        NEU       0.69      0.35      0.46      1825
        POS       0.80      0.95      0.87      5553

avg / total       0.76      0.77      0.75      8400

0.77369047619
0.7805
0.7825
0.745
0.784583333333


In [42]:
# test out sgd (1.e-6)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.55      0.56      0.56      1022
        NEU       0.51      0.45      0.48      1825
        POS       0.84      0.87      0.85      5553

avg / total       0.73      0.74      0.73      8400

0.739642857143
0.721
0.7575
0.713
0.7625


In [39]:
# test out sgd (1.e-5)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.62      0.59      0.60      1022
        NEU       0.61      0.43      0.50      1825
        POS       0.83      0.92      0.87      5553

avg / total       0.76      0.77      0.76      8400

0.773571428571
0.776
0.779
0.743
0.7925


In [36]:
# test out sgd (1.e-4)
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

#sgd()
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


print(accuracy_score(test_out[6400:8400], test_pred[6400:8400]))
print(accuracy_score(test_out[4400:6400], test_pred[4400:6400]))
print(accuracy_score(test_out[2400:4400], test_pred[2400:4400]))
print(accuracy_score(test_out[:2400], test_pred[:2400]))

             precision    recall  f1-score   support

        NEG       0.65      0.53      0.58      1022
        NEU       0.70      0.31      0.43      1825
        POS       0.79      0.96      0.87      5553

avg / total       0.75      0.77      0.74      8400

0.765595238095
0.77
0.7775
0.7325
0.779583333333


In [32]:
# test out sgd 
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()


#t(koken twit pos) - s
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


             precision    recall  f1-score   support

        NEG       0.86      0.05      0.10      1022
        NEU       0.60      0.00      0.01      1825
        POS       0.67      1.00      0.80      5553

avg / total       0.68      0.67      0.54      8400

0.667976190476


In [29]:
# test out sgd 
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()


#t(koken twit pos) - s
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


             precision    recall  f1-score   support

        NEG       0.70      0.27      0.39      1022
        NEU       0.71      0.09      0.16      1825
        POS       0.71      0.99      0.82      5553

avg / total       0.71      0.71      0.63      8400

0.705595238095


In [ ]:
# 실패
from konlpy.tag import Twitter
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

                

model = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2))), 
            ('mb',   SVC(kernel='linear')),
        ])


X = train_df.review
y = train_df.rating_cat

model.fit(X, y)

In [ ]:
# 최고기록 0.7730
from konlpy.tag import Twitter
pos_tagger = Twitter()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

                
model = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1,2))), 
            ('mb',   LogisticRegression()),
        ])


X = train_df.review
y = train_df.rating_cat

model.fit(X, y)

In [26]:
# 낮음
model = Pipeline([
            ('vect', TfidfVectorizer()), 
            ('mb', LogisticRegression(C=10.0)),
        ])

X = train_df.review
y = train_df.rating_cat

model.fit(X, y)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [12]:
train_df['review_nouns'][:10]

0                                    ['종합', '평점', '점']
1    ['원작', '칭송', '이유', '웹툰', '계', '자체', '질적', '저하'...
2    ['나름', '감동', '마음', '가슴', '배우', '연기', '김수현', '최고']
3    ['걸', '돈', '자신', '후회', '최악', '쓰레기', '영화', '김수현...
4               ['초반', '코미디', '후반', '액션', '결론', '코미디']
5                       ['김수현', '일', '처리', '처리', '절차']
6                                    ['원작', '어디', '팔']
7                        ['나름', '장면', '해도', '나름', '볼']
8                                ['님', '이영화', '클레멘타인']
9                    ['좀', '억지', '감동', '좀', '유치', '함']
Name: review_nouns, dtype: object

In [17]:
train_df['review_nouns'][:10].astype(str).apply(lambda x: ' '.join(x))

0                    [ ' 종 합 ' ,   ' 평 점 ' ,   ' 점 ' ]
1    [ ' 원 작 ' ,   ' 칭 송 ' ,   ' 이 유 ' ,   ' 웹 툰 ' ...
2    [ ' 나 름 ' ,   ' 감 동 ' ,   ' 마 음 ' ,   ' 가 슴 ' ...
3    [ ' 걸 ' ,   ' 돈 ' ,   ' 자 신 ' ,   ' 후 회 ' ,   ...
4    [ ' 초 반 ' ,   ' 코 미 디 ' ,   ' 후 반 ' ,   ' 액 션 ...
5    [ ' 김 수 현 ' ,   ' 일 ' ,   ' 처 리 ' ,   ' 처 리 ' ...
6                    [ ' 원 작 ' ,   ' 어 디 ' ,   ' 팔 ' ]
7    [ ' 나 름 ' ,   ' 장 면 ' ,   ' 해 도 ' ,   ' 나 름 ' ...
8            [ ' 님 ' ,   ' 이 영 화 ' ,   ' 클 레 멘 타 인 ' ]
9    [ ' 좀 ' ,   ' 억 지 ' ,   ' 감 동 ' ,   ' 좀 ' ,   ...
Name: review_nouns, dtype: object

In [7]:
from konlpy.tag import *
twitter = Twitter()

In [24]:
train_df['review_nouns'] = train_df['review'].apply(twitter.nouns)

In [1]:
train_df.to_csv('train_df.csv')

NameError: name 'train_df' is not defined

In [13]:
twitter.nouns('안녕하세요 저는 사람입니다. 사랑합니다.')

['저', '사람']

In [24]:
#input_file = '/elice/data/grading.input'#sys.argv[1] #'/elice/data/test.input' #
input_file = '/elice/data/test.input' 
output_file = 'submission.txt'#sys.argv[2] # 



with open(input_file) as fp:
    test_in = fp.read()

test_in = test_in.splitlines()
test_pred = model.predict(test_in[:8400])


with open(output_file, 'w') as fp:
    for _test_pred in test_pred:
        fp.write(_test_pred)
        fp.write('\n')
        
#t(koken twit pos) - s
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: X has 8400 features per sample; expecting 1862890

In [ ]:
from konlpy.tag import Twitter
pos_tagger = Twitter()

In [58]:
with open('/elice/data/test.output') as fp:
    test_out = fp.read()
    test_out = test_out.splitlines()

In [11]:
#t-l(koken twit pos)
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))


             precision    recall  f1-score   support

        NEG       0.64      0.53      0.58      1022
        NEU       0.62      0.42      0.50      1825
        POS       0.82      0.93      0.87      5553

avg / total       0.76      0.77      0.76      8400

0.773095238095


In [31]:
#t-l(c=10))
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))



             precision    recall  f1-score   support

        NEG       0.60      0.48      0.53      1022
        NEU       0.55      0.41      0.47      1825
        POS       0.81      0.91      0.85      5553

avg / total       0.73      0.75      0.73      8400

0.74630952381


In [59]:
#t-linear svc
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))

             precision    recall  f1-score   support

        NEG       0.61      0.58      0.60      1022
        NEU       0.58      0.44      0.50      1825
        POS       0.84      0.91      0.87      5553

avg / total       0.75      0.77      0.76      8400

0.768333333333


In [69]:
#t-linear svc (5)
print(classification_report(test_out[:8400],test_pred))
print(accuracy_score(test_out[:8400], test_pred))

             precision    recall  f1-score   support

        NEG       0.57      0.57      0.57      1022
        NEU       0.52      0.44      0.48      1825
        POS       0.84      0.88      0.86      5553

avg / total       0.74      0.75      0.74      8400

0.74619047619


In [7]:
len(test_pred)

4391